# 2D Aggregates Simulation
A simulation of 2D that demonstrates random motion of bacteria cells.<br><br>
Each cell is within an aggregate, even if it is the only cell that is in that aggregate. This allows the aggregates to add cells through collision and birth. Additionally, aggregates can lose cells when cells simply break apart or when cells die. The goal of this simulation is to provide information on how the aggregation of bacteria (specifically *P. aeruginosa*)

# Cell-level Parameters

|   Simulation Parameter   |  Description   |   Units   |  Variable Name   |
|:------|:-------|:-------|:--------|
|$p=(p_{x},p_{y})$ | Cell position in continuous $(x,y)$ space|$\mu$m|__cell.x__, __cell.y__|
|$\hat{p}$|Corresponding lattice position for $p$|--|obtained via __calculate_lattice_coords__|
|$\mu_R$|Maximal growth rate, given resource $R$|hr$^{-1}$|__cell.max_growth__|
|$d$| Death rate |hr$^{-1}$|__PROB_DEATH__ / __SIM_DT__|
|$\beta$|Separation rate|hr$^{-1}$|__CREATES_OWN_AGGREGATE_PROBABILITY__ / __SIM_DT__|
|$b$|Rate that cells leave an aggregate|hr$^{-1}$|__BREAK_PROBABILITY__ / __SIM_DT__|







## Aggregate-level Parameters

|   Simulation Parameter   |  Description   |   Units   |  Variable Name   |
|:------|:-------|:-------|:--------|
|$a^t$ | Total number of aggregates in the environment at time $t$|aggregates| -- |
|$c^t_i$ | Total number of cells in aggregate $i$ at time $t$ ("aggregate size")|cells|__aggregate.size__|
|$d_{agg}$ | Threshold radius for diffusive aggregation |$\mu m$| __COLLISION_DISTANCE__|
|$\rho$ | Diffusive aggregation rate|$hr^{-1}$|__MERGE_ON_COLLISION_PROB__ / __SIM_DT__|
|$D_A$ | Aggregate diffusion rate|$\mu m^{2} / hr$|__PROB_MOVE__ / __SIM_DT__|
|$k$ | Mean squared displacement of an aggregate in time interval $t$ |$\mu m$|not currently implemented (i.e. 10 in __dx__, __dy__ formula)|
|$c_{thresh}$ | Threshold aggregate size|cells|__SHEDDING_FLOOR__|
|$\theta$ | Direction/angle of aggregate diffusion|$rad$|__dir__|







## Environmental Parameters

|   Simulation Parameter   |  Description   |   Units   |  Variable Name   |
|:------|:-------|:-------|:--------|
|$(\hat{p}_{pos}, \hat{p}_{neg})$ | Positive and negative limits of simulation space|$\mu m$|__POS_LIM__, __NEG_LIM__|
|$m, n$ | Lattice dimensions (i.e. $m$ rows x $n$ columns)|--|(__POS_LIM__ - __NEG_LIM__)|
|$\Delta x_L$ | Lattice spatial step size|$\mu m$|__LATTICE_SPACING__|
|$\Delta t$| Simulation time step|$hr$|__SIM_DT__|
|$D_S$ | Diffusion coefficient for substrate $S$|$\mu$m$^{2}$ / $hr$|__lattice.diffusion_coefficient__|

## Resource Parameters 

|   Simulation Parameter   |  Description   |   Units   |  Variable Name   |
|:------|:-------|:-------|:--------|
|$\gamma_R$ | Yield coefficient for resource|--|__Resource.yield__ |
|$k_R$ | Half-maximal concentration of resource|$\mu$g/ml|__Resource.half_maximal__ |
|$\mu_R$ |Maximal growth rate|$hr^{-1}$|__MAX_GROWTH_RATE__|

## Added Chemical Parameters
Change to 'substrates' and state variable $S$?

Substrates that are not produced by bacteria

|   Simulation Parameter   |  Description   |   Units   |  Variable Name   |
|:------|:-------|:-------|:--------|
|$f_{qq}$ | Quorum-quenching factor|--|__AddedChemical.quorum_quenched_aggregation__|
|$f_{aa}$ | Anti-adhesion factor|--|__AddedChemical.anti_adhesion_factor__|
|$f_{sm}$ | Surface-modification factor|--|__AddedChemical.surface_modification_factor__|
|$f_{bs}$ | Bacteriostatic factor|--|__AddedChemical.bacteriostatic_factor__|
|$f_{bc}$ | Bactericidal factor|--|__AddedChemical.bacteriocidal_factor__|
|$k_S$ |Half-maximal concentration of substrate $S$|$\mu$g/ml|__AddedChemical.half_maximal__|
|$\lambda_S$ | Half-life for substrate $S$ - should be rate??||__AddedChemical.half_life__|
|$t_{S,in}$ | Time step at which chemical is first introduced|$hr$|__AddedChemical.intro_time_step__|


## Public Good Parameters
Use state variable $P$?

More broadly substrates that are produced by cells, namely public goods, but could adapt to be defense mechanisms in a multi-species/strain scenario

|   Simulation Parameter   |  Description   |   Units   |  Variable Name   |
|:------|:-------|:-------|:--------|
|$f_A$ | Aggregation factor||__PublicGood.aggregate_factor__|
|$f_g$ | Growth factor||__PublicGood.growth_factor__|
|$\alpha$ | Scale factor||__PublicGood.scale_factor__|
|$k_P$ | Half-maximal public good concentration|$\mu$g/ml|__PublicGood.half_maximal__|
|$\gamma_{P}$ | Yield coefficient of public good|--|__PublicGood.yield__|
|$\zeta$| Growth cost of public good production|$hr^{-1}$|__PublicGood.cost__|

## Cell Level Properties
* Birth
* Death
* Growth Rate

In [ ]:
mutable struct Cell
    x::Float64
    y::Float64
    prob_birth::Float64
    max_growth::Float64
end

## Cell Functions
- periodic_boundary(cell)
  - Loops the cell to the other side of the map when it crosses the boundaries

In [ ]:
function periodic_boundary(cell::Cell)
    if cell.x > POS_X
        cell.x = NEG_LIM + 1
    elseif cell.x < NEG_LIM
        cell.x = POS_LIM - 1
    end
    if cell.y > POS_LIM
        cell.y = NEG_LIM + 1
    elseif cell.y < NEG_LIM
        cell.y = POS_LIM - 1
    end
end

periodic_boundary (generic function with 1 method)

In [ ]:
function reflective_boundary(cell::Cell, dx, dy)
    if ((cell.x + dx) > POS_LIM_X)
        cell.x = POS_LIM_X - (dx + cell.x - POS_LIM_X)
    elseif ((cell.x + dx) < NEG_LIM)
        cell.x = NEG_LIM_X - (dx + cell.x - NEG_LIM_X)
    else
        cell.x += dx
    end
    if ((cell.y + dy) > POS_LIM_Y)
        cell.y = POS_LIM_Y - (dy + cell.y - POS_LIM_Y)
    elseif ((cell.y + dy) < negLim)
        cell.x = NEG_LIM_X - (dy + cell.y - NEG_LIM_Y)
    else
        cell.y += dy
    end
end

reflective_boundary (generic function with 1 method)

In [ ]:
function generate_newcell(parent_cell::Cell)
    x_diff = 2 * NEWCELL_MAX_DIST * rand() - NEWCELL_MAX_DIST
    y_diff = 2 * NEWCELL_MAX_DIST * rand() - NEWCELL_MAX_DIST
    newcell = Cell(
        parent_cell.x + x_diff,
        parent_cell.y + y_diff,
        parent_cell.prob_birth,
        parent_cell.max_growth
    )
    return newcell
end

generate_newcell (generic function with 1 method)

## Aggregate Level Properties
* Movement
* Color

In [ ]:
import ColorTypes
mutable struct Aggregate
    cells::Set{Cell}
    rgb::ColorTypes.RGBA
    size::Int32
end

## Aggregate Functions
- is_collision(aggregate1, aggregate2)
  - checks to see if there is a collision between two aggregates
- add_cell(aggregate, cell)
  - adds the cell to the aggregate
- del_cell(aggregate, cell)
  - removes the cell from the aggregate
- merge_aggregates(aggregate1, aggregate2)
  - adds all of one aggregates cells to another aggregate
direction


In [ ]:
function is_collision(aggregate::Aggregate, new_aggregate::Aggregate)
    for cell1 in aggregate.cells
        for cell2 in new_aggregate.cells
            if (sqrt((cell1.x - cell2.x)^2 + (cell1.y - cell2.y)^2) <= COLLISION_DISTANCE)
                return true
            end
        end
    end
    return false
end

is_collision (generic function with 1 method)

In [ ]:
function add_cell(aggregate::Aggregate, cell::Cell)
    push!(aggregate.cells, cell)
    aggregate.size += 1
end

add_cell (generic function with 1 method)

In [ ]:
function del_cell(aggregate::Aggregate, cell::Cell)
    delete!(aggregate.cells, cell)
    aggregate.size -= 1
end

del_cell (generic function with 1 method)

In [ ]:
function merge_aggregates(Aggregate_Map::Set{Aggregate}, aggregate::Aggregate, new_aggregate::Aggregate)
    # Completes a union of the aggregate.cells sets
    union!(aggregate.cells, new_aggregate.cells)
    aggregate.size += new_aggregate.size
    delete!(Aggregate_Map, new_aggregate)
end

merge_aggregates (generic function with 1 method)

In [ ]:
function break_apart(Aggregate_Map::Set{Aggregate}, aggregate::Aggregate, breakaway_cell::Cell)
    agg_rand_cell = rand(aggregate.cells)

    # The following if statements ensure that the cells move away from the random aggregate cell
    # in both the x and y directions
    x_mov = BREAKAWAY_DIST
    if breakaway_cell.x < agg_rand_cell.x
        x_mov = -BREAKAWAY_DIST
    end

    y_mov = BREAKAWAY_DIST
    if breakaway_cell.y < agg_rand_cell.y
        y_mov = -BREAKAWAY_DIST
    end
        
    breakaway_cell.x += x_mov
    breakaway_cell.y += y_mov
    
    # Create new aggregate with the cell as root
    new_aggregate = generate_new_aggregate(breakaway_cell)
    del_cell(aggregate, breakaway_cell)  
    push!(Aggregate_Map, new_aggregate)
end

break_apart (generic function with 1 method)

## Lattice
We use this struct to keep track of nutrient and cell concentrations. So far, there are two main instances of the lattice class in the simulation. For more information, read about Fick's Second Law.
* nutrient_lattice is a 2D array that keeps track of the amount of nutrient available in each location
* cell_density_lattice is a 2D array that keeps track of the density of cells in each location. This is directly mappable to the nutrient lattice.
### Lattice Properties
    * lattice_resolution: the dimensions of the lattice are lattice_resolution x lattice_resolutio
    * matrix: the matrix representing the lattice
    * diffusion_coefficient: proporitionality constant between the flux and the gradient of the concentration

In [ ]:
mutable struct Lattice
    lattice_resolution_x::Int64
    lattice_resolution_y::Int64
    matrix::Matrix{Float64}
    diffusion_coefficient::Int64
end

## Lattice Functions
- initalize_to_uniform(lattice)
  - initializes the lattice by setting all cells to the same value
- show_lattice(lattice)
  - visualizes the lattice
- calc_diffusion(lattice)
  - calculates the calculates the new resource lattice after diffusion
- calc_lattice_coords(x, y)
  - calculates the coordinates of a cell within the density lattice

In [ ]:
function initialize_to_uniform(lattice::Lattice, initial_value)
    lattice.matrix = zeros(Float64, lattice.lattice_resolution_y, lattice.lattice_resolution_x) .+ initial_value
    return lattice
end

initialize_to_uniform (generic function with 1 method)

In [ ]:
function initialize_to_random(lattice::Lattice, total_value)
    # Initialize matrix to zeros
    lattice.matrix = zeros(Float64, lattice.lattice_resolution_y, lattice.lattice_resolution_x)
    remaining_resource = total_value
    
    # Generate random values for each cell
    random_values = rand(lattice.lattice_resolution_y, lattice.lattice_resolution_x)
    
    # Normalize the random values so they sum to 1
    normalized_values = random_values / sum(random_values)
    
    # Scale the normalized values to sum to total_value
    lattice.matrix = normalized_values * total_value
    
    return lattice
end

initialize_to_random (generic function with 1 method)

In [ ]:
function initialize_added_chemical_lattice(lattice::Lattice, half_maximal::Float64)
    lattice.matrix = half_maximal .+ zeros(Float64, (lattice.lattice_resolution_y, lattice.lattice_resolution_x))
end

initialize_added_chemical_lattice (generic function with 1 method)

In [ ]:
using Plots

# Not to scale yet
function show_lattice(lattice::Lattice)
    matrix = lattice.matrix
    indices = findall(x -> x < CLIM[1], lattice.matrix)
    matrix[indices] .= CLIM[1]
    indices = findall(x -> x > CLIM[2], matrix)
    matrix[indices] .= CLIM[2]
    return heatmap(
        matrix',
        color=CMAP,
        xlabel="Mesh X Index",
        ylabel="Mesh Y Index",
        xlims=(0, lattice.lattice_resolution_x),
        ylims=(0, lattice.lattice_resolution_y),
        clim=CLIM,
        aspect_ratio=:equal,
    )
end

show_lattice (generic function with 1 method)

In [ ]:
function calculate_diffusion(lattice)
    
    y_flux_padded = zeros(Float64,lattice.lattice_resolution_y + 2, lattice.lattice_resolution_x)
    
    y_flux_padded[1,:] = lattice.matrix[2,:] .+ (2*INFLOW_GRADIENT*LATTICE_SPACING/lattice.diffusion_coefficient) 
    y_flux_padded[end,:] = lattice.matrix[end-1,:] .- (2*OUTFLOW_GRADIENT*LATTICE_SPACING/lattice.diffusion_coefficient) .* lattice.matrix[end, :]
    y_flux_padded[2:end-1,:] .= lattice.matrix
    
    y_flux = -lattice.diffusion_coefficient * (y_flux_padded[2:end,:] .- y_flux_padded[1:end-1,:])/LATTICE_SPACING
    
    dR = -((y_flux[2:end,:] .- y_flux[1:end-1,:])/LATTICE_SPACING)
    
    x_flux_padded = zeros(Float64,lattice.lattice_resolution_y, lattice.lattice_resolution_x + 2)
    
    x_flux_padded[:,1] = lattice.matrix[:,2] .+ (2*INFLOW_GRADIENT*LATTICE_SPACING/lattice.diffusion_coefficient) 
    x_flux_padded[:,end] = lattice.matrix[:,end-1] .- (2*OUTFLOW_GRADIENT*LATTICE_SPACING/lattice.diffusion_coefficient) .* lattice.matrix[:, end]
    x_flux_padded[:,2:end-1] .= lattice.matrix
    
    x_flux = -lattice.diffusion_coefficient * (x_flux_padded[:,2:end] .- x_flux_padded[:,1:end-1])/LATTICE_SPACING
    
    dR = dR - ((x_flux[:,2:end] .- x_flux[:,1:end-1])/LATTICE_SPACING)
    
    lattice.matrix .+= dR * RESOURCE_DT
    
end

calculate_diffusion (generic function with 1 method)

In [ ]:
function calculate_lattice_coords(x, y)
    # Calculate x and y coordinates in the lattice space based on limits and spacing
    x_coord = round(Int64, (x - NEG_LIM_X) / LATTICE_SPACING)
    y_coord = round(Int64, (y - NEG_LIM_Y) / LATTICE_SPACING)

    # Clamp the coordinates to ensure they stay within the matrix bounds
    x_coord = clamp(x_coord, 1, LATTICE_RESOLUTION_X)
    y_coord = clamp(y_coord, 1, LATTICE_RESOLUTION_Y)

    return (x_coord, y_coord)
end


calculate_lattice_coords (generic function with 1 method)

In [ ]:
function get_lattice_squares_per_micron(lattice_spacing)
    return round(Int, 1.0 / lattice_spacing)  # Convert 1 micron to lattice squares
end

get_lattice_squares_per_micron (generic function with 1 method)

# Added Chemicals
Here, the user can mimic real chemicals, or explore how theoretical chemicals could affect the growth rate of *_P. aeruginosa_*. All the chemicals represented by the AddedChemical struct should be chemicals that are not produced by the organisms that are currently being simulated in the system. These chemicals should be added by an outside agent.

## Aggregation Inhibition
This feature allows us to explore the potential effects of limiting the aggregation of *_P. aeruginosa_*
### Quorum-Quenching Chemicals
Quorum Quenching has been shown to be an effective mechanism for reducing aggregation in many bacteria. All the techniques that are modeled here have been successful at reducing *_P. aeruginosa_* aggregation.
- *_N_*-Decanoyl Cyclopentylamide (C10-CPA)
  - "Intervenes with the *_las_* and *_rhl_* system via inhibiting interaction between their response regulators and autoinducers."
  - Inhibits *_lasB-lacZ_* expression 80% of the time at 80 $\mu$M. How do I represent this in the simulation?
  - In the real experiment, C10-CPA does not seem to impact growth rate.

### Anti-Adhesives
- *_A. philippense_* crude extract
  - Reduced pre-formed biofilm formation by 56.54%
  - Reduced EPS by 66.73%
  - Also lowers growth rate though

### Surface Modification
- TODO: Find an example for Pseudomonas

## Antibiotics
### Bacteriostatic Antibiotics
- Some chemicals inhibit a bacteria's ability to reproduce 
- For the purpose of this simulation, we just used a basic model that is not based on a real world example

### Bactericidal Antibiotics
- Oral Ciprofloxacin is effective against *P. aeruginosa*.
  - Has a half life of 4 hours in plasma vs Pseudomonas' doubling rate of 2 hours

### AddedChemical Properties
* quorum_quenched_aggregation: float representing the proportion of aggregation to be inhibited
* anti_adhesion_factor: 
* surface_modification_factor:
* bacteriostatic_factor: 0<float<1 representing the factor to decrease the growth probability by
* bactericidal_factor: 1<float representing the factor to increase the death probability by
                                
* half_maximal: the half-maximal concentration of the chemical
* lattice: chemical concentration lattice
* intro_time_step: time step at which chemical is introduced to the system    
* half_life: If the chemical experiences a half-life, this is the duration of that half-life. The formula $$0.5^\frac{SIM_DT}{half_life}$$ will be used to find the decay factor during each iteration

In [ ]:
mutable struct AddedChemical
    quorum_quenched_aggregation_factor::Float64
    anti_adhesion_factor::Float64
    surface_modification_factor::Float64
    bacteriostatic_factor::Float64
    bactericidal_factor::Float64
    
    half_maximal::Float64
    lattice::Lattice
    intro_time_step::Float64
    half_life::Float64
end

## Commonly Used Chemicals

### C10-CPA

In [ ]:
function C10_CPA(intro_time_step, half_life)
    return AddedChemical(0.3, 0, 0, 0, 0, 100, Lattice(0,0,zeros(Float64,1,1),0), intro_time_step, half_life)
end

C10_CPA (generic function with 1 method)

### Ciprofloxacin

In [ ]:
# Common Antibiotic used on P. aeruginosa
function CIPROFLOXACIN(intro_time_step)
    return AddedChemical(0, 0, 0, 0, 200, 180, Lattice(0,0,zeros(Float64,1,1),0), intro_time_step, 4)
end

CIPROFLOXACIN (generic function with 1 method)

### Bacteriostatic Antibiotic

In [ ]:
function BACTERIOSTATIC(intro_time_step, half_life)
    return AddedChemical(0, 0, 0, 0.05, 0, 100, Lattice(0,0,zeros(Float64,1,1),0), intro_time_step, half_life)
end

BACTERIOSTATIC (generic function with 1 method)

# Public Good
The Public Good struct abstracts substances like siderophores or exoenzymes produced by aggregates themselves that increase the growth rate or aggregation of cells when cells come in contact with them. An example of this is the siderophore pyoverdine, produced by *Pseudomonas* to make the uptake and transport of iron more efficient, and is tied to increased virulence. It's benefits for *Pseudomonas* have been seen to level off with high saturation, but have proven short-term effects ("the growth rate increases almost linearly with PVD concentration, then sharply levels off") (Becker et. al, 2018).

In [ ]:
mutable struct PublicGood
    scale_factor::Float64
    half_maximal::Float64
    yield::Float64
    cost::Float64
    lattice::Lattice
end

# Resource
Resource refers to a nutrient-gradient modeling basic nutrients used by *Pseudomonas* for growth such as a carbon source, nitrogen molecules, or iron. Users can specify the initial distribution of nutrients across the environment, as well as describe the boundary conditions and rate of diffusion for nutrient in the simulation space.

In [ ]:
mutable struct Resource
    half_maximal::Float64
    yield::Float64
    lattice::Lattice
end

## Simulation Helper Functions

In [ ]:
# Redacted due to publication pending. Please reach out to learn more!

calculate_dx_dy (generic function with 1 method)

## Simulation Code

In [ ]:
# Redacted due to publication pending. Please reach out to learn more!

run (generic function with 2 methods)

In [76]:
function save_plots(Aggregate_Map, resource_lattice, time_step, photo_file)
    cell_movement = plot_agg(Aggregate_Map, time_step * 1.0)
    #resource_graph = show_lattice(resource_lattice)
    create_plots(cell_movement, photo_file)
end

save_plots (generic function with 1 method)

In [77]:
using Plots

function plot_agg(Aggregate_Map::Set{Aggregate}, CURR_T::Float64)
    # Create the base scatter plot
    p = Plots.scatter(legend=false, xlabel="X Microns", ylabel="Y Microns", hover=true, aspect_ratio=:equal)
    title!("Discrete Time: Time Step $CURR_T")
    xlims!(NEG_LIM_X, POS_LIM_X)
    ylims!(NEG_LIM_Y, POS_LIM_Y)

    # Compute dynamic axis range
    xrange = POS_LIM_X - NEG_LIM_X  # Width in microns
    yrange = POS_LIM_Y - NEG_LIM_Y  # Height in microns

    # Define a reasonable scaling constant (tweak if needed)
    base_marker_size = 200  # Adjust this constant if needed

    # Compute marker size dynamically so that 1 micron always looks the same
    scaling_factor = min(base_marker_size / xrange, base_marker_size / yrange)
    marker_size = scaling_factor  # Already scaled to 1 µm in the current axis range

    # Scatter plot each cell with the adjusted marker size
    for aggregate in Aggregate_Map
        for cell in aggregate.cells
            Plots.scatter!(p, [cell.x], [cell.y], markersize=marker_size, color=aggregate.rgb)
        end
    end

    return p
end


plot_agg (generic function with 1 method)

In [78]:
using Plots

function create_plots(p1, photo_file)
    plot(p1)
    Plots.savefig(photo_file)
end

create_plots (generic function with 1 method)

## Saving Parameters

In [80]:
function save_parameters()
    if SAVE_PARAMETERS
        dir = string("experiments/", TEST_DIRECTORY)
        txt_file = joinpath(dir, "parameters.txt")
        file = open(txt_file, "w")
        write(file, string("RUN_TIME: ", RUN_TIME, " seconds", "\n"))
        write(file, string("HOURS: ", HOURS, "\n"))
        write(file, string("NUM_RUNS: ", NUM_RUNS, "\n"))
        write(file, string("SIM_DT: ", SIM_DT, "\n"))
        write(file,string("RESOURCE_DT: ", RESOURCE_DT, "\n"))
        write(file, string("T_MAX: ", T_MAX, "\n"))
        write(file, string("INITIAL_AGGREGATES: ", INITIAL_AGGREGATES, "\n"))
        write(file, string("AGGREGATE_SIZE: ", AGGREGATE_SIZE, "\n"))
        write(file, string("PROB_MOVE: ", PROB_MOVE, "\n"))
        write(file, string("PROB_DEATH: ", PROB_DEATH, "\n"))
        write(file, string("NO_MERGE_ON_COLLISION_PROBABILITY: ", NO_MERGE_ON_COLLISION_PROBABILITY, "\n"))
        write(file, string("CREATES_OWN_AGG_PROBABILITY: ", CREATES_OWN_AGG_PROBABILITY, "\n"))
        write(file, string("BREAK_PROBABILITY: ", BREAK_PROBABILITY, "\n"))
        write(file, string("BREAKAWAY_DIST: ", BREAKAWAY_DIST, "\n"))
        write(file, string("MAX_GROWTH_RATE: ", MAX_GROWTH_RATE, "\n"))
        write(file, string("NEWCELL_MAX_DIST: ", NEWCELL_MAX_DIST, "\n"))
        write(file, string("COLLISION_DISTANCE: ", COLLISION_DISTANCE, "\n"))
        write(file, string("SHEDDING_FLOOR: ", SHEDDING_FLOOR, "\n"))
        write(file, string("MOV_MAGNITUDE: ", MOV_MAGNITUDE, "\n"))
        write(file, string("DIFFUSION_COEFFICIENT: ", DIFFUSION_COEFFICIENT, "\n"))
        write(file, string("BOUNDARY_CONDITION: ", BOUNDARY_CONDITION, "\n"))
        write(file, string("INFLOW_GRADIENT: ", INFLOW_GRADIENT, "\n"))
        write(file, string("OUTFLOW_GRADIENT: ", OUTFLOW_GRADIENT, "\n"))
        write(file, string("DENSITY_DEPENDENT_MOVEMENT_THRESHOLD: ", DENSITY_DEPENDENT_MOVEMENT_THRESHOLD, "\n"))
        write(file, string("CMAP: ", CMAP, "\n"))
        write(file, string("CLIM: ", CLIM, "\n"))
        write(file, string("POS_LIM_X: ", POS_LIM_X, "\n"))
        write(file, string("POS_LIM_Y: ", POS_LIM_Y, "\n"))
        write(file, string("NEG_LIM_X: ", NEG_LIM_X, "\n"))
        write(file, string("NEG_LIM_Y: ", NEG_LIM_Y, "\n"))
        write(file, string("LATTICE_SPACING: ", LATTICE_SPACING, "\n"))
        write(file, string("LATTICE_RESOLUTION_X: ", LATTICE_RESOLUTION_X, "\n"))
        write(file, string("LATTICE_RESOLUTION_Y: ", LATTICE_RESOLUTION_Y, "\n"))
        write(file, string("GENERATE_GRAPHS: ", GENERATE_GRAPHS, "\n"))
        write(file, string("TEST_DIRECTORY: ", TEST_DIRECTORY, "\n"))
        write(file, string("RESOURCE: ", RESOURCE, "\n"))
        write(file, string("INITIAL_RESOURCE: ", INITIAL_RESOURCE, "\n"))
        num = 1
        for ADDED_CHEMICAL in ADDED_CHEMICALS
            write(file, string("ADDED_CHEMICAL #", num, ": ", ADDED_CHEMICAL, "\n"))
            num += 1
        end
        write(file, string("PUBLIC_GOOD_ON: ", PUBLIC_GOOD_ON, "\n"))
        write(file, string("PUBLIC_GOOD: ", PUBLIC_GOOD, "\n"))
        write(file, string("PUBLIC_GOOD_DENSITY_THRESHOLD: ", PUBLIC_GOOD_DENSITY_THRESHOLD, "\n"))
        write(file, string("PUBLIC_GOOD_RESOURCE_THRESHOLD: ", PUBLIC_GOOD_RESOURCE_THRESHOLD, "\n"))
        close(file)
    end
end

save_parameters (generic function with 1 method)

## Saving Resource

In [82]:
# BELOW: RESOURCE MATRIX
#=function save_resource(resource_matrices::Vector{Matrix{Float64}}, run_name::String)
    filename = string(string("experiments/", TEST_DIRECTORY), "/resource")
    if !isdir(filename)
        mkdir(filename)
    end
    filename = string(string(filename, "/resource_"), run_name)
    open(filename, "w") do file
        for i in 1:length(resource_matrices)
            for row in eachrow(resource_matrices[i])
                println(file, join(row, ","))  
            end
            println(file)  # Blank line between matrices (for readability)
            println(file) 
        end
    end
end=#
# BELOW: Total Resource
function save_resource(total_resource_values::Vector{Float64}, run_name::String)
    filename = string("experiments/", TEST_DIRECTORY, "/resource")
    if !isdir(filename)
        mkdir(filename)
    end
    
    filename = string(filename, "/resource_", run_name)
    open(filename, "w") do file
        for total_resource in total_resource_values
            println(file, total_resource)
        end
    end
end

save_resource (generic function with 1 method)

## Saving Public Good

In [84]:
# BELOW: RESOURCE MATRIX
function save_public_good(public_matrices::Vector{Matrix{Float64}}, run_name::String)
    filename = string(string("experiments/", TEST_DIRECTORY), "/public_good_matrices")
    if !isdir(filename)
        mkdir(filename)
    end
    filename = string(string(filename, "/public_good_"), run_name)
    open(filename, "w") do file
        for i in 1:length(public_matrices)
            for row in eachrow(public_matrices[i])
                println(file, join(row, ","))  
            end
            println(file)  # Blank line between matrices (for readability)
            println(file) 
        end
    end
end

# Total public good
function save_total_public_good(total_public_good_values::Vector{Float64}, run_name::String)
    filename = string("experiments/", TEST_DIRECTORY, "/total_public_good")
    if !isdir(filename)
        mkdir(filename)
    end
    
    filename = string(filename, "/public_good_", run_name)
    open(filename, "w") do file
        for public_good in total_public_good_values
            println(file, public_good)
        end
    end
end

save_total_public_good (generic function with 1 method)

## Saving Aggregate Size Distribution

In [86]:
function save_aggregate_sizes(aggregate_size_matrix::Vector{Vector{Int32}}, run_name::String)
    filename = string("experiments/", TEST_DIRECTORY, "/aggregate_sizes")
    if !isdir(filename)
        mkdir(filename)
    end

    filename = string(filename, "/agg_sizes_", run_name)
    open(filename, "w") do file
        for size_row in aggregate_size_matrix
            println(file, join(size_row, ","))
        end
    end
end


save_aggregate_sizes (generic function with 1 method)

## Saving Cell Density

In [88]:
function save_cell_density(cell_count_values::Vector{Int}, run_name::String)
    filename = string("experiments/", TEST_DIRECTORY, "/cell_density")
    if !isdir(filename)
        mkdir(filename)
    end
    
    filename = string(filename, "/cell_density_", run_name)
    open(filename, "w") do file
        for cell_count in cell_count_values
            println(file, cell_count)
        end
    end
end

save_cell_density (generic function with 1 method)

## Saving Cell Locations

In [90]:
function save_cell_locations(cell_locations_over_time, run_name)
    filename = string("experiments/", TEST_DIRECTORY, "/cell_locations")
    if !isdir(filename)
        mkdir(filename)
    end

    filename = string(filename, "/cell_locations_", run_name)
    
    open(filename, "w") do file
        for (t, matrix) in enumerate(cell_locations_over_time)
            for row in eachrow(matrix)
                println(file, join(row, ","))
            end
            println(file, "")  # Separate snapshots with a newline
        end
    end
end

save_cell_locations (generic function with 1 method)

# Run the simuliation from here:

## Multithreading
If you would like to run this simulation using multiple threads, scroll to the bottom of the notebook, one box below the main simulation run box. You will see a short program that installs a jupyter kernel that allocates a user-specified number of threads for Julia. 8 threads are the recommended number based on our testing, but anything between 4-10 should could be efficient. After doing this, you may have to refresh the jupyter lab page. Click on the top right on the kernel (It should say something like "Julia v.#.#". Change the kernel to "Julia # Threads", or whatever you named the new kernel you installed. Now, run the simulation as you normally would!
## Data to save from the simulation
Choose whether you would like to save the simulation data to a database and whether you would like to display the output as graphs. If you would like to save the images of the graphs as a gif, open terminal/command line and type 
```
cd IBMAggregateModeling/graphs
```
```
python3 make_gif.py
```
to create a gif.
This enum allows the user to choose what type of boundary condition they would like to use.\nThis enum allows the user to choose what type of boundary condition they would like to use.\nThe choices are:
- Periodic:
  - Each cell that crosses boundary loops over to other side of map
- Outflow:
  - Each cell the crosses boundary is deleted from the aggregate (cell death)

In [93]:
############################### SETUP ####################################
@enum BoundaryConditions PERIODIC=1 OUTFLOW REFLECTIVE
##########################################################################

In [94]:
BOUNDARY_CONDITION = OUTFLOW::BoundaryConditions

OUTFLOW::BoundaryConditions = 2

In [149]:
using ThreadsX
using Base.Threads
using Dates

######################## SIMULATION PARAMETERS #############################
HOURS =  125 # Number of hours for the simulation to run
NUM_RUNS = 1
SIM_DT = 0.01
RESOURCE_DT_SCALING_FACTOR = 1000
RESOURCE_DT = SIM_DT/RESOURCE_DT_SCALING_FACTOR
T_MAX = round(Int, HOURS / SIM_DT) # Number of time steps is the number of hours divided by the SIM_DT
INITIAL_AGGREGATES = 4 # Initial number of aggregates
AGGREGATE_SIZE = 5 # Original aggregate size
PROB_MOVE = 0.9 # Probability the cell moves in any given iteration
PROB_DEATH = 0.1 # Probability cell dies over the course of an hour       # SET BACK TO 0.05
NO_MERGE_ON_COLLISION_PROBABILITY = 10 # SET BACK TO 0.95
CREATES_OWN_AGG_PROBABILITY = 0.5 # SET BACK TO 0.50
BREAK_PROBABILITY = 0
BREAKAWAY_DIST = 2
MAX_GROWTH_RATE = 0.25 # Max growth rate for 'species 1'
NEWCELL_MAX_DIST = 1 # The maximum distance in x and y direction away from the parent a new cell can form
COLLISION_DISTANCE = 1 # 2 - This is reflective of Pseudomonas A.'s dimensions (0.5-1 x 1-5 micrometers)
SHEDDING_FLOOR = 70 # Aggregate size at which shedding becomes likely
MOV_MAGNITUDE = 10 # The distance a cell can move in the x and y directions
DIFFUSION_COEFFICIENT = 15 # micrometer squared
RESOURCE_DIFFUSION = 2
DENSITY_DEPENDENT_MOVEMENT_THRESHOLD = 5 # Aggregate size at which movement starts slowing down
MAX_CELL_STACK_VOLUME = 4 # How many cells can be within a 1x1 micron space

# Lattice variables
CMAP=:Purples
CLIM=(-10,10)
POS_LIM_X = 6
POS_LIM_Y = 6
NEG_LIM_X = -6
NEG_LIM_Y = -6
LATTICE_SPACING = 0.075 # Size of individual lattice space - ensure that POS_LIM - NEG_LIM is divisble by this
LATTICE_RESOLUTION_Y = Int64(round((POS_LIM_X - NEG_LIM_X) / LATTICE_SPACING)) # lattice = (LATTICE_RESOLUTION_Y x LATTICE_RESOLUTION_X)
LATTICE_RESOLUTION_X = Int64(round((POS_LIM_Y - NEG_LIM_Y) / LATTICE_SPACING))

# Boundary conditions
BOUNDARY_CONDITION = OUTFLOW::BoundaryConditions
INFLOW_GRADIENT = 0*LATTICE_SPACING*LATTICE_SPACING
OUTFLOW_GRADIENT = 0*LATTICE_SPACING*LATTICE_SPACING

# Resource 
RESOURCE = Resource(0.3, 0.5, Lattice(LATTICE_RESOLUTION_X, LATTICE_RESOLUTION_Y, zeros(Float64,1,1), RESOURCE_DIFFUSION))
TOTAL_RESOURCE = 0  # This is total resource in the system
INITIAL_RESOURCE = TOTAL_RESOURCE/(LATTICE_RESOLUTION_X * LATTICE_RESOLUTION_Y)   # This is resource per cell
INITIALIZE_RESOURCE_TO_UNIFORM = true   # If true - uniform resource, if false - random resource

# Added Chemicals
ADDED_CHEMICALS = AddedChemical[]
# push!(ADDED_CHEMICALS, C10_CPA(1, 4)) # Use C10-CPA as a quorum quencher to reduce aggregation
#push!(ADDED_CHEMICALS, C10_CPA(11, 4)) # Use C10-CPA as a quorum quencher to reduce aggregation
# push!(ADDED_CHEMICALS, CIPROFLOXACIN(2)) # bactericidal antibiotic that kills P. aeruginosa (half-life = 4 hours)
# push!(ADDED_CHEMICALS, CIPROFLOXACIN(11)) # bactericidal antibiotic that kills P. aeruginosa (half-life = 4 hours)
# push!(ADDED_CHEMICALS, BACTERIOSTATIC(1, 4)) # Model for a bacteriostatic antibiotic's implementation
# push!(ADDED_CHEMICALS, BACTERIOSTATIC(11, 4)) # Model for a bacteriostatic antibiotic's implementation

# Public Goods
PUBLIC_GOOD_ON = false
PUBLIC_GOOD = PublicGood(0, 50, 10, 0.5, Lattice(LATTICE_RESOLUTION_X, LATTICE_RESOLUTION_Y, zeros(Float64,1,1),0))
PUBLIC_GOOD_DENSITY_THRESHOLD = 9
PUBLIC_GOOD_RESOURCE_THRESHOLD = 1450

# Data Collection variables
GENERATE_GRAPHS = true
SAVE_PARAMETERS = false
SAVE_AGG_SIZE_DIST = true # Not currently implemented
SAVE_AGGREGATE_GROWTH_RATE = false
SAVE_RESOURCE = false 
SAVE_PUBLIC_GOOD = false
SAVE_CELLS = false
SAVE_RESOURCE_GRANULARITY = 500 # Sim will save resource after every 'SAVE_RESOURCE_GRANULARITY' timesteps

TEST_DIRECTORY = "paper-figures/pace/4.1_growth-separation/final-tests/graphs" # This directory holding the simulation data is stored in 'experiments'
RUN_NAME = ""

############################################################################
# Set clock
start_time = time()

# Run sim
ThreadsX.foreach(1:NUM_RUNS) do i
    run_name = RUN_NAME * "test" * string(i)
    run(run_name)
end

# const N_THREADS = nthreads()
# println("Running on $N_THREADS threads")

# # Array to hold tasks
# tasks = Vector{Task}()

# for i in 1:NUM_RUNS
#     t = Threads.@spawn begin
#         run_name = RUN_NAME * "test" * string(i)
#         # Optional: minimal print to see progress
#         println("Starting run $i on thread $(threadid()) at $(Dates.now())")
        
#         # Run simulation
#         run(run_name)
        
#         # Free memory between runs
#         GC.gc()
        
#         println("Finished run $i at $(Dates.now())")
#     end
#     push!(tasks, t)
# end
# wait.(tasks)

# End clock
end_time = time()
RUN_TIME = round(end_time - start_time, digits=2)

# Save necessary parameters
save_parameters()

# 

In [ ]:
using Base.Threads

using IJulia
IJulia.installkernel("Julia 4 Threads", env=Dict(
    "JULIA_NUM_THREADS" => "4",
))